# Imports

In [1]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import meshplot as mp
from math import exp

import os
root_folder = os.getcwd()

# Load meshes

In [2]:
def normalize_unitbox(V):
	V = V - V.min()
	V = V / V.max()
	return V

In [3]:
## Load a mesh in OFF format
sphere_v, sphere_f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "sphere_s3.off"))
model_v, model_f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "bunny2.off"))
# model_v, model_f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "sphere.obj"))

model_v = normalize_unitbox(model_v)

# Precompute

In [4]:
# Get topology and cot laplacian
ev, fe, ef = igl.edge_topology(model_v, model_f)
L = igl.cotmatrix(model_v, model_f)
adj_f_list, NI = igl.vertex_triangle_adjacency(model_f, model_v.shape[0])
adj_f_list = [adj_f_list[NI[i]:NI[i+1]] for i in range(model_v.shape[0])]

arap_rhs = igl.arap_rhs(model_v, model_f, model_v.shape[1], igl.ARAP_ENERGY_TYPE_SPOKES_AND_RIMS)

faces_adj_edges = [fe[i] for i in adj_f_list]
faces_adj_vertices = [ev[i] for i in faces_adj_edges]

he_list = [faces_adj_vertices[i].reshape(-1, 2) for i in range(model_v.shape[0])]

w_vec_list = [np.array([[L[i, j] for i, j in f] for f in v]).reshape(-1,1) for v in faces_adj_vertices]

dv_list = [(model_v[he_list[i][:,1]] - model_v[he_list[i][:,0]]).reshape(-1,3).transpose() for i in range(model_v.shape[0])]

RAll = np.tile(np.eye(3), model_v.shape[0])

# Calculating function

In [5]:
def normalize_g_weights(N: list[np.array], sigma: float):
	mat = np.zeros((len(N), len(N)))
	mat = np.array([[exp(sigma * N[i][:].dot(N[j][:])) for j in range(len(N))] for i in range(len(N))])
	return sp.linalg.solve(mat, np.ones(len(N)))

In [6]:
def g_value(v, N, w, sigma):
	return sum(w[i] * exp(sigma * v.dot(N[i])) for i in range(len(N)))

def g_gradient(v, N, w, sigma):
	return sigma * sum(w[i] * N[i] * exp(sigma * v.dot(N[i])) for i in range(len(N)))

def g_hessian(v, N, w, sigma):
	return (sigma**2) * sum(w[i] * exp(sigma * v.dot(N[i])) * N[i].reshape((-1,1)) * N[i] for i in range(len(N)))


In [7]:
N = [
		np.array([1,0,0]), np.array([-1,0,0]),
		np.array([0,1,0]), np.array([0,-1,0]),
		np.array([0,0,1]), np.array([0,0,-1]),
	]
sigma = 8
mu = 1
lambda_value = 4
w = normalize_g_weights(N, sigma)

In [8]:
new_sphere_v = np.array([i * g_value(i, N, w, sigma) for i in sphere_v[:]])

In [18]:
import multiprocessing

def single_iteration(V, U, F, iterations):
	# Initialization
	e_ij_stars = [U[ev[i,1]] - U[ev[i,0]] for i in range(ev.shape[0])]
	nf_stars = igl.per_face_normals(U, F, np.array([1.0,0.0,0.0]))
	u = np.zeros([ev.shape[0], 3])
	
	# ADMM optimization
	for i in range(iterations):
		for f in range(F.shape[0]):
			g_grad = g_gradient(nf_stars[f,:], N, w, sigma)
			Hg = g_hessian(nf_stars[f,:], N, w, sigma)

			r_grad = np.zeros(3)
			Hr = np.zeros([3,3])
			for j in range(3):
				e = fe[f, j]
				v1 = ev[e, 0]
				v2 = ev[e, 1]
				w_ij = L[v1,v2]
				e_ij_star = e_ij_stars[e]
				r_grad += (lambda_value * w_ij * (e_ij_star.dot(nf_stars[f,:]) + u[f, j]) * e_ij_star)
				Hr += lambda_value * w_ij * (e_ij_star.reshape(-1,1) * e_ij_star.reshape(1,-1))

			# Newton step
			gn = (r_grad - g_grad)
			gn_newton = sp.linalg.lstsq(Hr - Hg, -gn)[0]

			# project gradient step
			pt = (np.eye(3) - nf_stars[f,:].reshape(-1,1) * nf_stars[f,:].reshape(1,-1))
			d = pt.dot(gn_newton)
			if d.dot(gn) > 0:
				d = -0.1 * pt.dot(gn)
		
			# Update normal
			nf_stars[f,:] += d
			nf_stars[f,:] /= np.linalg.norm(nf_stars[f,:])

		for e in range(ev.shape[0]):
			f1 = ef[e, 0]
			f2 = ef[e, 1]
			v1 = ev[e, 0]
			v2 = ev[e, 1]
			e_ij = U[v2,:] - U[v1,:]
			w_ij = L[v1,v2]

			A = np.eye(3)
			rhs = e_ij

			if f1 != -1:
				A += mu * nf_stars[f1,:].reshape(-1,1) * nf_stars[f1,:].reshape(1,-1)
				for k in range(3):
					if fe[f1, k] == e:
						rhs -= mu * nf_stars[f1,:] * u[f1, k]
			if f2 != -1:
				A += mu * nf_stars[f2,:].reshape(-1,1) * nf_stars[f2,:].reshape(1,-1)
				for k in range(3):
					if fe[f2, k] == e:
						rhs -= mu * nf_stars[f2,:] * u[f2, k]

			e_ij_stars[e] = sp.linalg.lstsq(A, rhs)[0]

		# Update u
		# TODO

	E_target_edges_rhs = np.zeros([V.shape[0], 3])
	for e in range(ev.shape[0]):
		v1 = ev[e, 0]
		v2 = ev[e, 1]
		w_ij = L[v1,v2]
		E_target_edges_rhs[v1,:] -= w_ij * e_ij_stars[e]
		E_target_edges_rhs[v2,:] += w_ij * e_ij_stars[e]


	# ARAP local step
	rotations = []
	for i in range(U.shape[0]):
		edge_starts = U[he_list[i][:,0]]
		edge_ends = U[he_list[i][:,1]]

		SB = (dv_list[i].dot(np.diag(w_vec_list[i].flatten()))).dot(edge_ends - edge_starts)

		u, s, vh = np.linalg.svd(SB)
		if np.linalg.det(u.dot(vh)) < 0:
			vh[2,:] *= -1
		RAll[:,i*3:(i+1)*3] = u.dot(vh).transpose()
		rotations.append(u.dot(vh).transpose())

	# ARAP global step
	Rcol = np.array([rot[j,i] for i in range(3) for j in range(3) for rot in rotations]).reshape(-1,1)
	# for b in range(V.shape[0]):
	# 	for i in range(3):
	# 		for j in range(3):
	# 			Rcol[j*3*V.shape[0] + i*V.shape[0] + b] = RAll[i, b*3 + j]
	# [rotations[i][0,0] for i in range(len(rotations))]
	# Rcol2 = np.concatenate(()

	Bcol = arap_rhs.dot(Rcol)
	for dim in range(V.shape[1]):
		Bc = (Bcol[dim*V.shape[0]:(dim+1)*V.shape[0]] + lambda_value * E_target_edges_rhs[:,dim].reshape(-1,1)) / (1 + lambda_value)

		bcc = np.array([V[F[0,0],dim]])
		Uc = igl.min_quad_with_fixed(L, Bc, np.array([0],dtype=int), bcc, sp.sparse.csr_matrix((0,0)), np.array([]), False)
		U[:,dim] = Uc[1]

In [19]:
import copy
U = copy.deepcopy(model_v)
for i in range(5):
	single_iteration(model_v, U, model_f, 1)

In [19]:
for i in range(5):
	single_iteration(model_v, U, model_f, 1)

In [31]:
U.shape

(6358, 3)

In [ ]:
U30 = copy.deepcopy(U)

In [ ]:
U20 = copy.deepcopy(U)

In [11]:
p = mp.plot(new_sphere_v, sphere_f, return_plot=True)

vertices = [new_sphere_v, sphere_v]

@mp.interact(mesh=[('displaced', 0), ('sphere', 1)])
def ff(mesh):
    mp.plot(vertices[mesh], sphere_f, plot=p)
p

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(Dropdown(description='mesh', options=(('displaced', 0), ('sphere', 1)), value=0), Output…

In [20]:
p = mp.plot(model_v, model_f, return_plot=True)

vertices = [model_v, U]

@mp.interact(mesh=[('before', 0), ('after', 1)])
def ff(mesh):
    mp.plot(vertices[mesh], model_f, plot=p)
p

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.2771663…

interactive(children=(Dropdown(description='mesh', options=(('before', 0), ('after', 1)), value=0), Output()),…